# 🚀 Fine-tuning Qwen2.5 con Axolotl su Colab

Questo notebook utilizza Axolotl per fare il fine-tuning del modello Qwen2.5-7B-Instruct su dati medici.

**NOTA**: Questo approccio è più stabile rispetto a LLaMA Factory per ambienti Colab.

### 0. **Verifica Ambiente Colab**

In [ ]:
# Verifica connessione internet
!curl -s -o /dev/null -w "%{http_code}" https://www.google.com

# Verifica GPU
import torch
print("GPU disponibile:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Nome GPU:", torch.cuda.get_device_name(0))
    print("Memoria GPU totale:", torch.cuda.get_device_properties(0).total_memory // 1024**3, "GB")

# Verifica RAM
!free -h

# Verifica CPU
!nproc

### 1. **Setup Ambiente**

In [ ]:
# Monta Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Rimuovi directory esistente per clonazione pulita
!rm -rf /content/axolotl_training

# Torna alla directory base
%cd /content

# Clona il repository
!git clone https://github.com/Attapulgite999/prove.git

# Entra nella directory del progetto Axolotl
%cd prove/axolotl_training

### 2. **Installazione Axolotl**

In [ ]:
# Aggiorna pip
!pip install --upgrade pip

# Installa PyTorch compatibile con CUDA
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Installa Axolotl
!pip install -U axolotl[deepspeed]

# Installa dipendenze aggiuntive
!pip install transformers datasets accelerate peft bitsandbytes

# Verifica installazione
!axolotl --version

### 3. **Avvio Training**

In [ ]:
# Avvia il training con Axolotl
!accelerate launch -m axolotl.cli.train config/qwen_axolotl.yaml

### 4. **Conversione in GGUF**

In [ ]:
# Installa llama.cpp per la conversione
!pip install llama-cpp-python

# Clona llama.cpp se necessario
!git clone https://github.com/ggerganov/llama.cpp.git

# Converti il modello in GGUF
# Nota: modifica il path in base al tuo output directory
!python llama.cpp/convert-hf-to-gguf.py ./outputs/medical_qwen_axolotl/merged --outfile ./outputs/medical_qwen_axolotl/model.gguf --outtype q4_k_m

### 5. **Download del Modello**

In [ ]:
# Scarica il file GGUF
from google.colab import files
files.download('./outputs/medical_qwen_axolotl/model.gguf')